In [4]:
import os
import logging
import time
import glob
import datetime
import argparse
import numpy as np
from scipy.io import loadmat, savemat
from sklearn.feature_extraction import image
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from glob import glob

import cv2
from PIL import Image

In [5]:
def write_in_file(metrics, name_metric, name_dataset):
    with open(f"./metrics/{name_dataset}_{name_metric}.txt", "w") as file:
        for metric in metrics:
            file.write(str(metric) + '\n')

In [3]:
dataset_name = 'Crystal_focus_0_dose_180'
original_dir = f'./data/train/{dataset_name}/'
denoised_dir = './test/b2u_Crystal/'

original_ext = os.path.splitext(os.listdir(original_dir)[0])[1]
denoised_ext = os.path.splitext(os.listdir(denoised_dir)[0])[1]

file_list_original = sorted(glob(os.path.join(os.path.abspath(original_dir), '*' + original_ext)))
file_list_denoised = sorted(glob(os.path.join(os.path.abspath(denoised_dir), '*' + denoised_ext)))

os.makedirs('./metrics/', exist_ok=True)

['/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G113_C10_R0___06.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G113_C11_R0___05.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G122_C11_R0___07.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G140_C11_R0___00.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G20_C11_R0___01.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G211_C11_R0___03.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G40_C11_R0___02.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G59_C10_R1___17.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_180/G59_C11_R1___18.tif',
 '/home/aleksandra/SSD/vscode/Blind2Unblind/data/train/Crystal_focus_0_dose_1

In [ ]:
msd_value = []
psnr_value = []
ssim_value = []

for original, denoised in zip(file_list_original, file_list_denoised):
    original = cv2.imread(original, 0)
    denoised = cv2.imread(denoised, 0)

    msd = mean_squared_error(original, denoised)
    psnr = peak_signal_noise_ratio(denoised, original)
    ssim = structural_similarity(original, denoised)

    msd_value.append(msd)
    psnr_value.append(psnr)
    ssim_value.append(ssim)

In [6]:
av_msd = np.mean(msd_value)
av_psnr = np.mean(psnr_value)
av_ssim = np.mean(ssim_value)

write_in_file(msd_value, 'msd', name_dataset)
write_in_file(psnr_value, 'psnr', name_dataset)
write_in_file(ssim_value, 'ssim', name_dataset)

print(f'average msd for {dataset_name} = {round(av_msd, 4)}\naverage psnr for {dataset_name} = {round(av_psnr, 4)}\naverage ssim for {dataset_name} = {round(av_ssim, 4)}\n')



NameError: name 'msd_value' is not defined

In [ ]:
def ssim(prediction, target):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2
    img1 = prediction.astype(np.float64)
    img2 = target.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())
    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
    ssim_map = ((2 * mu1_mu2 + C1) *
                (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                       (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(target, ref):
    '''
    calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    img1 = np.array(target, dtype=np.float64)
    img2 = np.array(ref, dtype=np.float64)
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1[:, :, i], img2[:, :, i]))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')


def calculate_psnr(target, ref, data_range=255.0):
    img1 = np.array(target, dtype=np.float32)
    img2 = np.array(ref, dtype=np.float32)
    diff = img1 - img2
    psnr = 10.0 * np.log10(data_range**2 / np.mean(np.square(diff)))
    return psnr

: 